# 🚀 Deploy Qwen3 32B FP8 Model on Amazon SageMaker AI 

## Introduction: [Qwen3-32B-FP8](https://huggingface.co/Qwen/Qwen3-32B-FP8)

`Qwen3-32B-FP8` is part of the [latest generation of Qwen language models](https://qwenlm.github.io/blog/qwen3/) with:

- **Total Parameters**: 30.8B parameters
- **Number of Paramaters (Non-Embedding)**: 31.2B parameters
- **Architecture Details**:
    - 64 layers
    - 64 attention heads for queries and 8 for key/values (GQA)
    - context length of 32,768 tokens (expandable to 131,072 with YaRN)

### Note on FP8
For convenience and performance, we have provided fp8-quantized model checkpoint for Qwen3, whose name ends with -FP8. The quantization method is fine-grained fp8 quantization with block size of 128. You can find more details in the quantization_config field in config.json

### Key Features
1. Hybrid Thinking Modes:

- Thinking Mode: Enables step-by-step reasoning for complex problems
Non-Thinking Mode: Provides quick responses for simpler queries
Seamless switching between modes for optimal performance

2. Strong Capabilities:

- Advanced reasoning and problem-solving
- Excellent instruction following
- Enhanced agent capabilities for tool integration
- Support for 119+ languages and dialects

3. Model Architecture:

- MoE architecture enabling efficient parameter usage
- Only activates ~10% of parameters during inference
- Optimized for both performance and computational efficiency
  
This model represents a significant advancement in open-source language models, offering competitive performance while maintaining efficient resource utilization through its MoE architecture. It's particularly well-suited for deployment in production environments where both performance and cost efficiency are crucial considerations.
Let's get started deploying one of the most capable open-source reasoning models available today!

In [ ]:
%pip install -Uq sagemaker

### Setup

In [ ]:
import sagemaker
import sys
import boto3
import logging
import time
from sagemaker.session import Session
from sagemaker.s3 import S3Uploader

print(sagemaker.__version__)

In [ ]:
try:
    boto_region = boto3.Session().region_name
    sagemaker_session = sagemaker.session.Session(boto_session=boto3.Session(region_name=boto_region))
    role = sagemaker.get_execution_role()
    sagemaker_client = boto3.client("sagemaker",region_name=boto_region)
    prefix = sagemaker.utils.unique_name_from_base("DEMO")
    
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

## Setup your SageMaker Real-time Endpoint 
### Create a SageMaker endpoint configuration

We begin by creating the endpoint configuration and set MinInstanceCount to 0. This allows the endpoint to scale in all the way down to zero instances when not in use. See the [notebook example for SageMaker AI endpoint scale down to zero](https://github.com/aws-samples/sagemaker-genai-hosting-examples/tree/02236395d44cf54c201eefec01fd8da0a454092d/scale-to-zero-endpoint).

There are a few parameters we want to setup for our endpoint. We first start by setting the variant name, and instance type we want our endpoint to use. In addition we set the *model_data_download_timeout_in_seconds* and *container_startup_health_check_timeout_in_seconds* to have some guardrails for when we deploy inference components to our endpoint. In addition we will use Managed Instance Scaling which allows SageMaker to scale the number of instances based on the requirements of the scaling of your inference components. We set a *MinInstanceCount* and *MinInstanceCount* variable to size this according to the workload you want to service and also maintain controls around cost. Lastly, we set *RoutingStrategy* for the endpoint to optimally tune how to route requests to instances and inference components for the best performance.

The suggested instance types to host the Qwen3-32B-FP8 model can be `ml.g5.24xlarge`, `ml.g6.12xlarge`, `ml.g6e.12xlarge`.

In [ ]:
# Set an unique endpoint config name
endpoint_config_name = f"{prefix}-endpoint-config"
print(f"Demo endpoint config name: {endpoint_config_name}")

# Set varient name and instance type for hosting
variant_name = "AllTraffic"
instance_type = "ml.g6.12xlarge"
model_data_download_timeout_in_seconds = 3600
container_startup_health_check_timeout_in_seconds = 3600

min_instance_count = 0 # Minimum instance must be set to 0
max_instance_count = 1

sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=role,
    ProductionVariants=[
        {
            "VariantName": variant_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_in_seconds,
            "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_in_seconds,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": min_instance_count,
                "MaxInstanceCount": max_instance_count,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ],
)

### Create the SageMaker endpoint
Next, we create our endpoint using the above endpoint config

In [ ]:
# Set a unique endpoint name
endpoint_name = f"{prefix}-endpoint"
print(f"Demo endpoint name: {endpoint_name}")

sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

In [ ]:
sagemaker_session.wait_for_endpoint(endpoint_name)

## Download the model from Hugging Face and upload the model artifacts on Amazon S3
If you are deploying a model hosted on the HuggingFace Hub, you must specify the `option.model_id=<hf_hub_model_id>` configuration. When using a model directly from the hub, we recommend you also specify the model revision (commit hash or branch) via `option.revision=<commit hash/branch>`. 

Since model artifacts are downloaded at runtime from the Hub, using a specific revision ensures you are using a model compatible with package versions in the runtime environment. Open Source model artifacts on the hub are subject to change at any time. These changes may cause issues when instantiating the model (updated model artifacts may require a newer version of a dependency than what is bundled in the container). If a model provides custom model (modeling.py) and/or custom tokenizer (tokenizer.py) files, you need to specify option.trust_remote_code=true to load and use the model.

In this example, we will demonstrate how to download your copy of the model from huggingface and upload it to an s3 location in your AWS account, then deploy the model with the downloaded model artifacts to an endpoint.  

In [ ]:
HF_MODEL_ID = "Qwen/Qwen3-32B-FP8"

base_name = HF_MODEL_ID.split('/')[-1].replace('.', '-').lower()
model_lineage = HF_MODEL_ID.split("/")[0]
base_name

**Best Practices**:
>
> **Store Models in Your Own S3 Bucket**
For production use-cases, always download and store model files in your own S3 bucket to ensure validated artifacts. This provides verified provenance, improved access control, consistent availability, protection against upstream changes, and compliance with organizational security protocols.
>

First, let's download the model artifact data from HuggingFace.

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os
import sagemaker
import jinja2

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path(".")
local_model_path.mkdir(exist_ok=True)
model_name = HF_MODEL_ID
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.safetensors", "*.bin", "*.txt"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

### Upload model files to S3
SageMaker AI allows us to provide [uncompressed](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-uncompressed.html) files. Thus, we directly upload the folder that contains model files to s3
> **Note**: The default SageMaker bucket follows the naming pattern: `sagemaker-{region}-{account-id}`

In [ ]:
s3_model_prefix = (
    "hf-large-models/model_qwen3_32B_FP8"  # folder within bucket where model artifact will go
)

In [ ]:
model_artifact = sagemaker_session.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"We will set option.s3url={model_artifact}")

### Configure Model Serving Properties

Now we'll create a `serving.properties` file that configures how the model will be served. 

In [ ]:
# Create the directory that will contain the configuration files
from pathlib import Path

model_dir = Path('config-Qwen3-32B-FP8')
model_dir.mkdir(exist_ok=True)

**Best Practices**:
>
>**Separate Configuration from Model Artifacts**
> The LMI container supports separating configuration files from model artifacts. While you can store serving.properties with your model files, placing configurations in a distinct S3 location allows for better management of all your configurations files.
>
> **Note**: When your model and configuration files are in different S3 locations, set `option.model_id=<s3_model_uri>` in your serving.properties file, where `s3_model_uri` is the S3 object prefix containing your model artifacts. SageMaker AI will automatically download the model files by looking at the S3URI in model_id

In [ ]:
config = f"""engine=Python
option.async_mode=true
option.rolling_batch=disable
option.entryPoint=djl_python.lmi_vllm.vllm_async_service
option.model_loading_timeout=1500
fail_fast=true
option.max_rolling_batch_size=8
option.trust_remote_code=false
option.model_id={model_artifact}
option.enable_auto_tool_choice=true
option.tool_call_parser=hermes
"""
with open("config-Qwen3-32B-FP8/serving.properties", "w") as f:
    f.write(config)

#### Optional configuration files

(Optional) You can also specify a `requirements.txt` to install additional libraries. 
We update vllm to version vllm==0.8.5 for FP8 quantized model files

In [ ]:
%%writefile config-Qwen3-32B-FP8/requirements.txt
vllm==0.8.5

### Upload config files to S3
Here we will upload our config files to a different path to keep model files and config separate.

In [ ]:
from sagemaker.s3 import S3Uploader

sagemaker_default_bucket = sagemaker_session.default_bucket()

config_files_uri = S3Uploader.upload(
    local_path="config-Qwen3-32B-FP8",
    desired_s3_uri=f"s3://{sagemaker_default_bucket}/lmi/{base_name}/config-files"
)

print(f"code_model_uri: {config_files_uri}")

## Configure Model Container and Instance

For deploying Qwen3-32B-FP8, we'll use:
- **LMI (Deep Java Library) Inference Container**: A container optimized for large language model inference
> **Note**: The region in the container URI should match your AWS region.

In [ ]:
CONTAINER_VERSION = '0.33.0-lmi15.0.0-cu128'
image_uri = "763104351884.dkr.ecr.{}.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128".format(sagemaker_session.boto_session.region_name, CONTAINER_VERSION)
print(image_uri)

## Create SageMaker Model

Now we'll create a SageMaker Model object that combines our:
- Container image (LMI)
- Model artifacts (configuration files)
- IAM role (for permissions)

This step defines the model configuration but doesn't deploy it yet. The Model object represents the combination of:

1. **Container Image** (`image_uri`): DJL Inference optimized for LLMs
2. **Model Data** (`model_data`): points to our configuration files in S3
3. **IAM Role** (`role`): Permissions for model execution

### Required Permissions
The IAM role needs:
- S3 read access for model artifacts
- CloudWatch permissions for logging
- ECR permissions to pull the container

In [ ]:
from sagemaker.utils import name_from_base

qwen3_32b_fp8_model={
    "Image": image_uri,
    'ModelDataSource': {
                'S3DataSource': {
                    'S3Uri': f"{config_files_uri}/", # Specify the S3 URI for your config files
                    'S3DataType': 'S3Prefix',
                    'CompressionType': 'None',
                }
            },
    "Environment": {
        "SAGEMAKER_MODEL_SERVER_WORKERS": "1",
    },
}

model_name = name_from_base(base_name, short=True)

# create SageMaker Model
sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    Containers=[qwen3_32b_fp8_model],
)

> **Note**: Here S3 URI points to the configuration files S3 location

## Deploy Model to SageMaker Endpoint
We can now create the Inference Components which will deployed on the endpoint that you specify. Please note here that you can provide a SageMaker model or a container to specification. If you provide a container, you will need to provide an image and artifactURL as parameters. In this example we set it to the model name we prepared in the cells above. You can also set the `ComputeResourceRequirements` to supply SageMaker what should be reserved for each copy of the inference component. You can also set the copy count of the number of Inference Components you would like to deploy. These can be managed and scaled as the capabilities become available. 

Note that in this example we set the `NumberOfAcceleratorDevicesRequired` to a value of `4`. By doing so we reserve 4 accelerators for each copy of this inference component so that we can use tensor parallel. 

In [ ]:
from datetime import datetime

prefix = sagemaker.utils.unique_name_from_base("DEMO")

inference_component_name_qwen = f"{prefix}-IC-qwen3-32b-fp8-{datetime.now().strftime('%y%m%d-%H%M%S')}"
variant_name = "AllTraffic"

sagemaker_client.create_inference_component(
    InferenceComponentName=inference_component_name_qwen,
    EndpointName=endpoint_name,
    VariantName=variant_name,
    Specification={
        "ModelName": model_name,
        "ComputeResourceRequirements": {
            "NumberOfAcceleratorDevicesRequired": 4,
            "MinMemoryRequiredInMb": 104096,
        },
    },
    RuntimeConfig={"CopyCount": 1},
)

In [ ]:
import time
# Let's see how much it takes
start_time = time.time()
while True:
    desc = sagemaker_client.describe_inference_component(
        InferenceComponentName=inference_component_name_qwen
    )
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)
total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

#### Invoke endpoint with boto3
Now you can invoke the endpoint with boto3 `invoke_endpoint` or `invoke_endpoint_with_response_stream` runtime api calls. If you have an existing endpoint, you don't need to recreate the `predictor` and can follow below example to invoke the endpoint with an endpoint name.

Note that based on the [Qwen3 hugging face page description](https://huggingface.co/Qwen/Qwen3-30B-A3B), by default, Qwen3 has thinking capabilities enabled, similar to QwQ-32B. This means the model will use its reasoning abilities to enhance the quality of generated responses. In this mode, the model will generate think content wrapped in a \<think\>...\</think\> block, followed by the final response. For thinking mode, use Temperature=0.6, TopP=0.95, TopK=20, and MinP=0 (the default setting in generation_config.json).

It also allows a hard switch to strictly disable the model's thinking behavior, aligning its functionality with the previous Qwen2.5-Instruct models. This mode is particularly useful in scenarios where disabling thinking is essential for enhancing efficiency. For non-thinking mode, we suggest using Temperature=0.7, TopP=0.8, TopK=20, and MinP=0.

**Advanced Usage**: You can also switch Between `Thinking` and `Non-Thinking` Modes via User Input
Qwen3 provides a soft switch mechanism that allows users to dynamically control the model's behavior when `enable_thinking=True`. Specifically, you can add `/think` and `/no_think` to user prompts or system messages to switch the model's thinking mode from turn to turn. The model will follow the most recent instruction in multi-turn conversations.

In [ ]:
import boto3
import json

sagemaker_runtime_client = boto3.client("sagemaker-runtime", region_name=boto_region)

prompt = {
    'messages':[
    {"role": "user", "content": "How many R are in STRAWBERRY? Keep your answer and explanation short!"}
],
    'temperature':0.7,
    'top_p':0.8,
    'top_k':20,
    'max_tokens':512,
}
response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    InferenceComponentName=inference_component_name_qwen,
    ContentType="application/json",
    Body=json.dumps(prompt)
)
response_dict = json.loads(response['Body'].read().decode("utf-8"))
response_content = response_dict['choices'][0]['message']['content']
print(response_content)

In [ ]:
# switch to no thinking using chat_template_kwargs
prompt = {
    'messages':[
    {"role": "user", "content": "How many R are in STRAWBERRY? Keep your answer and explanation short!"}
],
    'temperature':0.7,
    'top_p':0.8,
    'top_k':20,
    'max_tokens':512,
    "chat_template_kwargs": {"enable_thinking": False},
}
response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    InferenceComponentName=inference_component_name_qwen,
    ContentType="application/json",
    Body=json.dumps(prompt)
)
response_dict = json.loads(response['Body'].read().decode("utf-8"))
response_content = response_dict['choices'][0]['message']['content']
print(response_content)

#### Streaming

In [ ]:
body = {
    'messages':[
        {"role": "user", "content": "How many R are in STRAWBERRY? Keep your answer and explanation short!"},
    ],
    'temperature':0.9,
    'max_tokens':512,
    'stream': True,
}


In [ ]:
import json
import time

# Create SageMaker Runtime client
sagemaker_runtime_client = boto3.client("sagemaker-runtime", region_name=boto_region)

## Add your endpoint here 
# endpoint_name = "DEMO-1749690961-f05b-endpoint"

# Invoke the model
response_stream = sagemaker_runtime_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType="application/json",
    InferenceComponentName=inference_component_name_qwen,
    Body=json.dumps(body)
)

first_token_received = False
ttft = None
token_count = 0
start_time = time.time()

print("Response:", end=' ', flush=True)
full_response = ""

for event in response_stream['Body']:
    if 'PayloadPart' in event:
        chunk = event['PayloadPart']['Bytes'].decode()
        
        try:
            # Handle SSE format (data: prefix)
            if chunk.startswith('data: '):
                data = json.loads(chunk[6:])  # Skip "data: " prefix
            else:
                data = json.loads(chunk)
            
            # Extract token based on OpenAI format
            if 'choices' in data and len(data['choices']) > 0:
                if 'delta' in data['choices'][0] and 'content' in data['choices'][0]['delta']:
                    token_count += 1
                    token_text = data['choices'][0]['delta']['content']
                                    # Record time to first token
                    if not first_token_received:
                        ttft = time.time() - start_time
                        first_token_received = True
                    full_response += token_text
                    print(token_text, end='', flush=True)
        
        except json.JSONDecodeError:
            continue
            
# Print metrics after completion
end_time = time.time()
total_latency = end_time - start_time

print("\n\nMetrics:")
print(f"Time to First Token (TTFT): {ttft:.2f} seconds" if ttft else "TTFT: N/A")
print(f"Total Tokens Generated: {token_count}")
print(f"Total Latency: {total_latency:.2f} seconds")
if token_count > 0 and total_latency > 0:
    print(f"Tokens per second: {token_count/total_latency:.2f}")

# Clean up

Make sure to delete the endpoint and other artifacts that were created to avoid unnecessary cost. You can also go to SageMaker AI console to delete all the resources created in this example.

In [ ]:
sagemaker_client.delete_inference_component(InferenceComponentName=inference_component_name_qwen)
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)